# ModelCheckpoint와 Earlystopping callback함수를 이용한 틴더의 relationship 성공 여부 예측 

<span style="color: blue; font-size: 18pt">
0. 라이브러리 임포트하기
</span>

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping

import pandas as pd
import numpy
import os
import tensorflow as tf

#seed 값 설정
numpy.random.seed(3)
tf.random.set_seed(3)

<span style="color: blue; font-size: 18pt">
1. 데이터셋을 pandas로 읽어오기
</span>

In [2]:
df_train = pd.read_csv("C:/Users/user/Downloads/final/5_Refined Dataset_(1).csv")


- 이번 딥러닝 프로젝트에 사용하는 데이터셋은 <span style="color: red">"5_Refined Dataset_(1)"</span> <span style="color: red">"5_Refined Dataset_(2)"</span>로 이루어져 있습니다. 

- 모델을 학습시킬 때는 <span style="color: red">"5_Refined Dataset_(1)"</span>에서 7:3을 나눠 학습 후 정확도를 측정합니다.

- y값이 존재하지 않는 <span style="color: red">"5_Refined Dataset_(2)"</span> 데이터셋은 마지막에 완성된 모델을 통한 예측 시에 사용합니다.


In [3]:
df_train

,ID,Answer,Count,Percentage,It became a relationship,Segment type_gender,Segment type_mobile,Segment type_university,Segment type_web,"Segment Description_Meridian, Idaho",Segment Description_SegDiscrip_Others
0,292890.8970,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
1,292887.9870,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
2,292894.0656,0,499.173606,0.225255,0,1.0,0.0,0.0,0.0,1.0,0.0
3,292887.1180,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
4,292893.6561,0,455.925963,0.211360,0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1891,292887.5496,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
1892,292881.6932,0,1203.190399,0.312360,0,0.0,1.0,0.0,0.0,1.0,0.0
1893,292900.8499,0,806.378820,0.488025,0,1.0,0.0,0.0,0.0,1.0,0.0
1894,292893.8600,0,1149.529381,0.488984,0,1.0,0.0,0.0,0.0,1.0,0.0


<span style="color: blue; font-size: 18pt">
2. sample() 함수 사용하기
</span>

In [4]:
df = df_train.sample(frac=0.15)

In [5]:
type(df)

pandas.core.frame.DataFrame

In [6]:
1896 * 0.15

284.4

In [7]:
len(df)

284

In [8]:
df

,ID,Answer,Count,Percentage,It became a relationship,Segment type_gender,Segment type_mobile,Segment type_university,Segment type_web,"Segment Description_Meridian, Idaho",Segment Description_SegDiscrip_Others
1220,292889.7537,0,1068.588264,0.348446,0,1.0,0.0,0.0,0.0,1.0,0.0
1384,293025.0000,1,3.000000,0.176000,1,0.0,0.0,1.0,0.0,0.0,1.0
800,292901.5645,0,529.876873,0.301602,0,1.0,0.0,0.0,0.0,1.0,0.0
1285,292883.8073,0,1002.631104,0.260338,1,0.0,1.0,0.0,0.0,1.0,0.0
1121,292887.1281,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
815,293185.0000,0,0.000000,0.000000,0,0.0,0.0,1.0,0.0,0.0,1.0
393,292887.0599,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
990,292890.8937,0,982.303666,0.357338,1,0.0,1.0,0.0,0.0,1.0,0.0
840,292896.6694,0,488.145272,0.250428,0,1.0,0.0,0.0,0.0,1.0,0.0


<span style="color: blue; font-size: 18pt">
3. 딥러닝을 위한 x, y 데이터 분리하기
</span>

In [9]:
dataset = df.values
X = dataset[:, [0, 1, 2, 3, 5, 6, 7, 8, 9, 10]]
Y = dataset[:, 4]

In [10]:
X

array([[2.92889754e+05, 0.00000000e+00, 1.06858826e+03, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.93025000e+05, 1.00000000e+00, 3.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [2.92901564e+05, 0.00000000e+00, 5.29876873e+02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [2.92890894e+05, 0.00000000e+00, 9.82303666e+02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.92896669e+05, 0.00000000e+00, 4.88145272e+02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.92882182e+05, 0.00000000e+00, 7.96829155e+02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [11]:
Y

array([0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [12]:
print("입력 데이터 개수 : ", len(X), "출력 데이터 개수 : ",len(Y) )

입력 데이터 개수 :  284 출력 데이터 개수 :  284


<span style="color: blue; font-size: 18pt">
4. 딥러닝 모델링 및 컴파일
</span>

<span style="color: magenta; font-size: 14pt">
5.1 모델 설정: 활성화함수로 시그모이드를 활용한 모델 설정
</span>

In [13]:
model = Sequential()
model.add(Dense(20,  input_dim=10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

<span style="color: magenta; font-size: 14pt">
5.2 모델 컴파일: loss='binary_crossentropy' 이용한 모델 컴파일
</span>

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

<span style="color: blue; font-size: 18pt">
5. 딥러닝 실행
</span>

<span style="color: magenta; font-size: 14pt">
5.1 딥러닝 모니터링 및 실행 중단을 위한 환경 설정하기
</span>

1) epoch 마다 딥러닝 모델을 저장할 폴더 만들기

In [15]:
from datetime import datetime
# 모델 저장 폴더 만들기

log_dir= datetime.now().strftime("%Y%m%d-%H%M%S")
dir_name = str(log_dir)

if not os.path.exists(datetime.now().strftime("%Y%m%d-%H%M%S") ):
    os.mkdir(dir_name)

In [16]:
dir_name

'20231217-192628'

In [17]:
os.chdir(dir_name)

In [18]:
os.getcwd()

'C:\\Users\\user\\Downloads\\final\\20231217-192628'

2. ModelCheckpoint( ) 함수를 이용한 모델 업데이트 과정을 체크하면서 저장하기

In [19]:
modelpath="./{epoch:02d}-{val_loss:.4f}.hdf5"

In [20]:
modelpath

'./{epoch:02d}-{val_loss:.4f}.hdf5'

In [21]:
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', save_weights_only=True, 
                               verbose=1, save_best_only=True)

In [22]:
checkpointer

3) EarlyStopping( ) 함수를 이용하여 학습 자동 중단 과정 설정하기

In [23]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

In [24]:
early_stopping_callback

<span style="color: magenta; font-size: 14pt">
5.2 딥러닝 모델 실행하기
</span>

train dataset과 test dataset을 7:3으로 설정해 실행

In [25]:
# 모델 실행 과정에서 홀드 아웃 교차 검증(Holdout Cross Validation) 사용
history = model.fit(X, Y, validation_split=0.3, epochs=3000, batch_size=500, verbose=2, 
                    callbacks=[checkpointer,early_stopping_callback])

Epoch 1/3000

Epoch 1: val_loss improved from inf to 23383.94531, saving model to .\01-23383.9453.hdf5
1/1 - 1s - loss: 25805.1992 - accuracy: 0.3232 - val_loss: 23383.9453 - val_accuracy: 0.3140 - 1s/epoch - 1s/step
Epoch 2/3000

Epoch 2: val_loss improved from 23383.94531 to 20520.73047, saving model to .\02-20520.7305.hdf5
1/1 - 0s - loss: 23050.8008 - accuracy: 0.3232 - val_loss: 20520.7305 - val_accuracy: 0.3140 - 92ms/epoch - 92ms/step
Epoch 3/3000

Epoch 3: val_loss improved from 20520.73047 to 17627.40625, saving model to .\03-17627.4062.hdf5
1/1 - 0s - loss: 20225.9727 - accuracy: 0.3232 - val_loss: 17627.4062 - val_accuracy: 0.3140 - 94ms/epoch - 94ms/step
Epoch 4/3000

Epoch 4: val_loss improved from 17627.40625 to 14713.64844, saving model to .\04-14713.6484.hdf5
1/1 - 0s - loss: 17371.4473 - accuracy: 0.3232 - val_loss: 14713.6484 - val_accuracy: 0.3140 - 97ms/epoch - 97ms/step
Epoch 5/3000

Epoch 5: val_loss improved from 14713.64844 to 11782.61816, saving model to .\05-1

Epoch 45/3000

Epoch 45: val_loss did not improve from 76.13375
1/1 - 0s - loss: 440.4909 - accuracy: 0.6768 - val_loss: 319.6901 - val_accuracy: 0.6860 - 60ms/epoch - 60ms/step
Epoch 46/3000

Epoch 46: val_loss did not improve from 76.13375
1/1 - 0s - loss: 324.5404 - accuracy: 0.6768 - val_loss: 127.8569 - val_accuracy: 0.6860 - 99ms/epoch - 99ms/step
Epoch 47/3000

Epoch 47: val_loss did not improve from 76.13375
1/1 - 0s - loss: 127.0583 - accuracy: 0.6768 - val_loss: 350.4368 - val_accuracy: 0.3140 - 56ms/epoch - 56ms/step
Epoch 48/3000

Epoch 48: val_loss did not improve from 76.13375
1/1 - 0s - loss: 328.8734 - accuracy: 0.3232 - val_loss: 454.9766 - val_accuracy: 0.3140 - 76ms/epoch - 76ms/step
Epoch 49/3000

Epoch 49: val_loss did not improve from 76.13375
1/1 - 0s - loss: 432.0713 - accuracy: 0.3232 - val_loss: 151.8266 - val_accuracy: 0.3721 - 51ms/epoch - 51ms/step
Epoch 50/3000

Epoch 50: val_loss did not improve from 76.13375
1/1 - 0s - loss: 130.6710 - accuracy: 0.3535 -


Epoch 90: val_loss did not improve from 36.26489
1/1 - 0s - loss: 160.3579 - accuracy: 0.6768 - val_loss: 282.3383 - val_accuracy: 0.6860 - 61ms/epoch - 61ms/step
Epoch 91/3000

Epoch 91: val_loss did not improve from 36.26489
1/1 - 0s - loss: 287.4455 - accuracy: 0.6768 - val_loss: 298.3039 - val_accuracy: 0.6860 - 61ms/epoch - 61ms/step
Epoch 92/3000

Epoch 92: val_loss did not improve from 36.26489
1/1 - 0s - loss: 303.9164 - accuracy: 0.6768 - val_loss: 217.0269 - val_accuracy: 0.6860 - 60ms/epoch - 60ms/step
Epoch 93/3000

Epoch 93: val_loss did not improve from 36.26489
1/1 - 0s - loss: 220.2693 - accuracy: 0.6768 - val_loss: 47.8796 - val_accuracy: 0.6860 - 67ms/epoch - 67ms/step
Epoch 94/3000

Epoch 94: val_loss did not improve from 36.26489
1/1 - 0s - loss: 46.1534 - accuracy: 0.6768 - val_loss: 474.7482 - val_accuracy: 0.3140 - 105ms/epoch - 105ms/step
Epoch 95/3000

Epoch 95: val_loss did not improve from 36.26489
1/1 - 0s - loss: 456.7807 - accuracy: 0.3232 - val_loss: 524

Epoch 136/3000

Epoch 136: val_loss did not improve from 28.45392
1/1 - 0s - loss: 24.8029 - accuracy: 0.4747 - val_loss: 246.2241 - val_accuracy: 0.3140 - 53ms/epoch - 53ms/step
Epoch 137/3000

Epoch 137: val_loss did not improve from 28.45392
1/1 - 0s - loss: 234.4024 - accuracy: 0.3232 - val_loss: 53.7558 - val_accuracy: 0.2907 - 131ms/epoch - 131ms/step
Epoch 138/3000

Epoch 138: val_loss did not improve from 28.45392
1/1 - 0s - loss: 44.5549 - accuracy: 0.3384 - val_loss: 209.1427 - val_accuracy: 0.6860 - 49ms/epoch - 49ms/step
Epoch 139/3000

Epoch 139: val_loss did not improve from 28.45392
1/1 - 0s - loss: 213.1136 - accuracy: 0.6768 - val_loss: 311.8691 - val_accuracy: 0.6860 - 78ms/epoch - 78ms/step
Epoch 140/3000

Epoch 140: val_loss did not improve from 28.45392
1/1 - 0s - loss: 318.9063 - accuracy: 0.6768 - val_loss: 309.7626 - val_accuracy: 0.6860 - 60ms/epoch - 60ms/step
Epoch 141/3000

Epoch 141: val_loss did not improve from 28.45392
1/1 - 0s - loss: 316.7652 - accurac

1/1 - 0s - loss: 359.9740 - accuracy: 0.6768 - val_loss: 159.3321 - val_accuracy: 0.6860 - 63ms/epoch - 63ms/step
Epoch 182/3000

Epoch 182: val_loss did not improve from 22.28883
1/1 - 0s - loss: 162.7016 - accuracy: 0.6768 - val_loss: 249.0186 - val_accuracy: 0.3140 - 48ms/epoch - 48ms/step
Epoch 183/3000

Epoch 183: val_loss did not improve from 22.28883
1/1 - 0s - loss: 240.5898 - accuracy: 0.3232 - val_loss: 343.8490 - val_accuracy: 0.3140 - 62ms/epoch - 62ms/step
Epoch 184/3000

Epoch 184: val_loss improved from 22.28883 to 19.71486, saving model to .\184-19.7149.hdf5
1/1 - 0s - loss: 334.2021 - accuracy: 0.3232 - val_loss: 19.7149 - val_accuracy: 0.3256 - 84ms/epoch - 84ms/step
Epoch 185/3000

Epoch 185: val_loss did not improve from 19.71486
1/1 - 0s - loss: 16.0338 - accuracy: 0.4596 - val_loss: 183.7086 - val_accuracy: 0.6860 - 59ms/epoch - 59ms/step
Epoch 186/3000

Epoch 186: val_loss did not improve from 19.71486
1/1 - 0s - loss: 187.8694 - accuracy: 0.6768 - val_loss: 251.


Epoch 226: val_loss did not improve from 11.19642
1/1 - 0s - loss: 229.8529 - accuracy: 0.6768 - val_loss: 182.3179 - val_accuracy: 0.6860 - 80ms/epoch - 80ms/step
Epoch 227/3000

Epoch 227: val_loss did not improve from 11.19642
1/1 - 0s - loss: 187.1618 - accuracy: 0.6768 - val_loss: 51.8684 - val_accuracy: 0.6860 - 49ms/epoch - 49ms/step
Epoch 228/3000

Epoch 228: val_loss did not improve from 11.19642
1/1 - 0s - loss: 52.8782 - accuracy: 0.6768 - val_loss: 352.1253 - val_accuracy: 0.3140 - 60ms/epoch - 60ms/step
Epoch 229/3000

Epoch 229: val_loss did not improve from 11.19642
1/1 - 0s - loss: 345.2945 - accuracy: 0.3232 - val_loss: 340.8232 - val_accuracy: 0.3140 - 62ms/epoch - 62ms/step
Epoch 230/3000

Epoch 230: val_loss did not improve from 11.19642
1/1 - 0s - loss: 334.2153 - accuracy: 0.3232 - val_loss: 44.3617 - val_accuracy: 0.6860 - 49ms/epoch - 49ms/step
Epoch 231/3000

Epoch 231: val_loss did not improve from 11.19642
1/1 - 0s - loss: 45.2077 - accuracy: 0.6768 - val_lo

Epoch 272/3000

Epoch 272: val_loss did not improve from 7.75772
1/1 - 0s - loss: 40.6494 - accuracy: 0.6768 - val_loss: 42.0399 - val_accuracy: 0.6860 - 60ms/epoch - 60ms/step
Epoch 273/3000

Epoch 273: val_loss did not improve from 7.75772
1/1 - 0s - loss: 43.4890 - accuracy: 0.6768 - val_loss: 101.0440 - val_accuracy: 0.3140 - 65ms/epoch - 65ms/step
Epoch 274/3000

Epoch 274: val_loss did not improve from 7.75772
1/1 - 0s - loss: 100.2273 - accuracy: 0.3232 - val_loss: 64.1054 - val_accuracy: 0.6860 - 46ms/epoch - 46ms/step
Epoch 275/3000

Epoch 275: val_loss did not improve from 7.75772
1/1 - 0s - loss: 66.2363 - accuracy: 0.6768 - val_loss: 72.7537 - val_accuracy: 0.6860 - 81ms/epoch - 81ms/step
Epoch 276/3000

Epoch 276: val_loss did not improve from 7.75772
1/1 - 0s - loss: 75.1561 - accuracy: 0.6768 - val_loss: 21.7816 - val_accuracy: 0.3140 - 49ms/epoch - 49ms/step
Epoch 277/3000

Epoch 277: val_loss did not improve from 7.75772
1/1 - 0s - loss: 22.1924 - accuracy: 0.3232 - va

Epoch 318/3000

Epoch 318: val_loss did not improve from 7.75772
1/1 - 0s - loss: 24.7688 - accuracy: 0.6768 - val_loss: 481.0868 - val_accuracy: 0.3140 - 65ms/epoch - 65ms/step
Epoch 319/3000

Epoch 319: val_loss did not improve from 7.75772
1/1 - 0s - loss: 477.5666 - accuracy: 0.3232 - val_loss: 551.5624 - val_accuracy: 0.3140 - 53ms/epoch - 53ms/step
Epoch 320/3000

Epoch 320: val_loss did not improve from 7.75772
1/1 - 0s - loss: 547.1453 - accuracy: 0.3232 - val_loss: 199.2328 - val_accuracy: 0.3140 - 63ms/epoch - 63ms/step
Epoch 321/3000

Epoch 321: val_loss did not improve from 7.75772
1/1 - 0s - loss: 199.6084 - accuracy: 0.3232 - val_loss: 238.7630 - val_accuracy: 0.6860 - 66ms/epoch - 66ms/step
Epoch 322/3000

Epoch 322: val_loss did not improve from 7.75772
1/1 - 0s - loss: 246.7506 - accuracy: 0.6768 - val_loss: 449.7950 - val_accuracy: 0.6860 - 52ms/epoch - 52ms/step
Epoch 323/3000

Epoch 323: val_loss did not improve from 7.75772
1/1 - 0s - loss: 464.0376 - accuracy: 0.6

<span style="color: magenta; font-size: 14pt">
5.3 딥러닝 실행 후 loss, val_loss, accuracy, val_accuracy 출력
</span>

In [26]:
print(history.history['loss'])

[25805.19921875, 23050.80078125, 20225.97265625, 17371.447265625, 14496.755859375, 11605.0224609375, 8697.3330078125, 5773.95654296875, 2834.61181640625, 73.06314849853516, 1209.0518798828125, 2104.206787109375, 2788.58642578125, 3287.701171875, 3623.462646484375, 3814.849365234375, 3878.400146484375, 3828.604248046875, 3678.204345703125, 3438.44921875, 3119.27587890625, 2729.502197265625, 2276.951904296875, 1768.5792236328125, 1210.5604248046875, 608.4008178710938, 108.70218658447266, 973.190673828125, 1317.614501953125, 1208.857177734375, 709.318603515625, 75.76697540283203, 347.4460144042969, 501.2731628417969, 547.8349609375, 496.8948059082031, 357.45550537109375, 137.78988647460938, 351.2075500488281, 488.6673278808594, 213.0182647705078, 212.402587890625, 393.81353759765625, 466.58673095703125, 440.4909362792969, 324.5403747558594, 127.058349609375, 328.8733825683594, 432.0712585449219, 130.67100524902344, 261.704345703125, 451.1344299316406, 531.1568603515625, 511.6508483886719,

In [27]:
print(history.history['val_loss'])

[23383.9453125, 20520.73046875, 17627.40625, 14713.6484375, 11782.6181640625, 8835.416015625, 5872.31689453125, 2893.03173828125, 76.13375091552734, 1179.52099609375, 2048.98046875, 2713.70849609375, 3198.48583984375, 3524.593994140625, 3710.468017578125, 3772.1728515625, 3723.78369140625, 3577.67578125, 3344.77685546875, 3034.737548828125, 2656.122802734375, 2216.534423828125, 1722.7254638671875, 1180.6947021484375, 595.7916870117188, 130.22198486328125, 1004.9199829101562, 1353.9307861328125, 1243.610595703125, 737.2045288085938, 78.30704498291016, 342.1703796386719, 491.5643615722656, 536.7695922851562, 487.2733154296875, 351.8177185058594, 138.436279296875, 373.65478515625, 512.91796875, 232.91563415527344, 210.83761596679688, 387.02630615234375, 457.6950378417969, 432.3303527832031, 319.69012451171875, 127.85688781738281, 350.43682861328125, 454.9765930175781, 151.82664489746094, 258.5738220214844, 442.55145263671875, 520.2612915039062, 501.2983703613281, 394.5309753417969, 208.14

In [28]:
print(history.history['accuracy'])  

[0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.3636363744735718, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.3232323229312897, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.3232323229312897, 0.3232323229312897, 0.34343433380126953, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.6767676472663879, 0.3232323229312897, 0.3232323229312897, 0.3535353541374206

In [29]:
print(history.history['val_accuracy']) 

[0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.3372093141078949, 0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.3139534890651703, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.3139534890651703, 0.3139534890651703, 0.3488371968269348, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.6860465407371521, 0.3139534890651703, 0.3139534890651703, 0.3720930218696594, 0.6860465407371521,

<span style="color: blue; font-size: 18pt">
6. 딥러닝 평가
</span>

In [31]:
# model.evaluate() 사용하여 성능 평가 
print('● 데이터 개수 : ', len(X), '개')
ev = model.evaluate(X, Y, batch_size=1)  
print('● [ loss 오차 , accuracy 정확도] =' , ev) 

● 데이터 개수 :  284 개
284/284 [==============================] - 1s 2ms/step - loss: 295.5367 - accuracy: 0.6796
● [ loss 오차 , accuracy 정확도] = [295.5366516113281, 0.6795774698257446]


In [32]:
print("\n Test Accuracy : %.4f" % (model.evaluate(X, Y, batch_size=10)[1])) 

29/29 [==============================] - 0s 2ms/step - loss: 295.5365 - accuracy: 0.6796

 Test Accuracy : 0.6796
